In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('attacks.csv', encoding='latin-1', index_col= False)
#Teniamos un error al leer el tipo de datos

In [3]:
df.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [4]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [5]:
### We will eliminate all the rows  and columns that are empty (near, empty)at the end that are empty
df = df[df.isnull().sum(axis=1) < 10]
df.dropna(thresh = 1000, axis=1, inplace=True)

In [6]:
#We will check for original order and see if we can use that for indexing our data. 
#We are only missing some value 1
df = df.set_index(df['original order'].astype(int).apply(lambda x: x-1))
df.drop('original order', axis=1, inplace=True)


In [7]:
df.index.name = None

In [8]:
###change year to a date,type
df['Year'] =  pd.to_datetime(df['Year'], format='%Y', errors = 'ignore')
df['Year']

6302    2018
6301    2018
6300    2018
6299    2018
6298    2018
        ... 
5          0
4          0
3          0
2          0
1          0
Name: Year, Length: 6295, dtype: object

In [9]:
#let's check if case number, case number.1 and case number.2 are the same
cases = df[['Case Number','Case Number.1','Case Number.2']].copy()

cases['diff']= np.where((df['Case Number'] == df['Case Number.1'])& (df['Case Number'] == df['Case Number.2']) & (df['Case Number.1'] == df['Case Number.2']), 'no change', 'diff')
cases.loc[cases['diff'] == 'diff']

,Case Number,Case Number.1,Case Number.2,diff
6268,2018.04.03,2018.04.02,2018.04.03,diff
6185,2017.07.20.a,2017/07.20.a,2017.07.20.a,diff
6158,2017.05.06,2017.06.06,2017.05.06,diff
6085,2016.09.15,2016.09.16,2016.09.15,diff
5988,2016.01.24.b,2015.01.24.b,2016.01.24.b,diff
5968,2015.12.23,2015.11.07,2015.12.23,diff
5963,2015.10.28.a,2015.10.28,2015.10.28.a,diff
5912,2015.07-10,2015.07.10,2015.07.10,diff
5742,2014.05.04,2013.05.04,2014.05.04,diff
2780,1967.07.05,1967/07.05,1967.07.05,diff


In [10]:
#Change values, 2/3 or best fit
df.loc[5912,'Case Number'] = df.loc[5912,'Case Number.1']
df.loc[2507,'Case Number'] = df.loc[2507,'Case Number.1']
df.loc[1877,'Case Number'] = df.loc[1877,'Case Number.1']
df.loc[1353,'Case Number'] = df.loc[1353,'Case Number.1']
df.loc[1027,'Case Number'] = df.loc[1027,'Case Number.1']
df.loc[814,'Case Number'] = df.loc[814,'Case Number.1']
df.loc[525,'Case Number'] = df.loc[525,'Case Number.1']
df.loc[358,'Case Number'] = df.loc[358,'Case Number.1']
df.loc[293,'Case Number'] = df.loc[293,'Case Number.1']


In [11]:
#Ahora podemos eliminar las columnas Case Number.1 y Case Number.2
df.drop(['Case Number.1', 'Case Number.2'], axis=1, inplace=True)
#Clean case number (..,-'/
df['Case Number']= df['Case Number'].str.replace('\.\.|\.|\'|\/|\s|\-','.', regex = True)
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href
6302,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
6301,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
6300,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
6299,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
6298,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [12]:
#We will check if we can reduce href and href formula to only one column
refs = df[['href','href formula']].copy()

refs['diff']= np.where(df['href'] == df['href formula'], 'no change', 'diff')
refs.loc[refs['diff'] == 'diff']

,href,href formula,diff
6252,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6206,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6171,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6169,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6161,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6134,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6068,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6061,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
6026,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5978,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff


In [13]:
refs['href'] = refs['href'].str.replace(".*(?=http://)", "")
refs['href formula'] = refs['href formula'].str.replace(".*(?=http://)", "")

In [14]:
#this fixes the double Url that we had in some entries
df['href'] = df['href'].str.replace(".*(?=http://)", "")
df['href formula'] = df['href formula'].str.replace(".*(?=http://)", "")


In [15]:
refs['diff']= np.where(df['href'] == df['href formula'], 'no change', 'diff')
refs.loc[refs['diff'] == 'diff']
#But we still have a lot of differente entries

,href,href formula,diff
6061,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5950,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5901,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5817,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5816,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5809,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5808,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5787,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5757,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff
5719,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,diff


In [16]:
#remplazar valores correctors de la otra columna
df.loc[5631,'href formula'] = df.loc[5631,'href']
df.loc[3058,'href formula'] = df.loc[3058,'href']
df.loc[3057,'href formula'] = df.loc[3057,'href']

In [17]:
#####We could take just the filename, and then add de case number to the url.
## so we will try to have just  the file name.pdf
refs['href'] = df['href'].str.extract("([^-]+.pdf$)")
refs['href formula'] = df['href formula'].str.extract("([^-]+.pdf$)")

In [18]:
####Let's clean Date so then we can clean Case number
df['Date'] = df['Date'].str.replace("Reported", "")


In [19]:
df['Date'] = pd.to_datetime( df['Date'], format='%d-%b-%Y', errors = 'ignore')
df['Date'] = pd.to_datetime( df['Date'], format='%b-%Y', errors = 'ignore')

In [20]:
##R solo aparece cuándo es reportado, pero existen casos R1 y R2 y hay que cabiarlos al formato a,b,c,..

#case['Case Number'] = df['Case Number'].str.replace("R1", "a")
#case['Case Number'] = df['Case Number'].str.replace("R2", "b")
#case['Case Number'] = df['Case Number'].str.replace("R3", "c")
#case['Case Number'] = df['Case Number'].str.replace('/' , ".")
#case['Case Number'] = df['Case Number'].str.replace("-", ".")
                                                    
#case[case['Case Number'].str.contains("R")]


In [21]:
######looking at the file name format, it has the case number and the victim/place
pdf = df.copy()
pdf['file'] = pdf['pdf'].str.extract("([^-]+.pdf$)")
pdf['case'] = pdf['pdf'].str.replace("(-[^-]+.pdf$)", "")
refs['files']=pdf['file']

In [22]:
###Comparing the end of the reference with the name of the pdf file
refs['diff']= np.where(refs['href'] == pdf['file'], 'no change', 'change')
refs.loc[refs['diff'] == 'change']

,href,href formula,diff,files
6272,NaN,NaN,change,NaN
6248,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,change,2017.12.30.Jones.pdf
6196,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,change,2017.07.31.pdf
6147,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,change,2017.04.17.a Brouwer.pdf
6129,NaN,NaN,change,NaN
5926,NaN,NaN,change,NaN
5904,NaN,NaN,change,NaN
5888,NaN,NaN,change,NaN
5817,NaN,Roberson.pdf,change,Roberson.pdf
5816,Bandy.pdf,"Olsson, pdf",change,"Olsson, pdf"


In [23]:
###it looks like 'href formula' is more like 'pdf' than 'href'
df.drop('href', axis=1, inplace=True)
refs.drop('href', axis=1, inplace=True)
refs['diff']= np.where(refs['href formula'] == refs['files'], 'no change', 'change')

In [24]:
refs.loc[refs['diff'] == 'change']

,href formula,diff,files
6272,NaN,change,NaN
6248,http://sharkattackfile.net/spreadsheets/pdf_di...,change,2017.12.30.Jones.pdf
6196,http://sharkattackfile.net/spreadsheets/pdf_di...,change,2017.07.31.pdf
6147,http://sharkattackfile.net/spreadsheets/pdf_di...,change,2017.04.17.a Brouwer.pdf
6129,NaN,change,NaN
5926,NaN,change,NaN
5904,NaN,change,NaN
5888,NaN,change,NaN
5737,NaN,change,NaN
5728,NaN,change,NaN


In [25]:
###in the Type Section after investigating, some consider boating a provoked attack, but 
##we don´t know how we will use the data so we will leave it like that
df['Type'] = df['Type'].str.replace(r"Boating|Boatomg", "Boat", regex =True)
#Sea disasters are unprovoked https://data.opendatasoft.com/explore/dataset/global-shark-attack%40public-us/information/
#But that database uses 5 categories so we will leave those 5


In [26]:
####Species####
dic ={'\d*\sm\s':'' ,'\d\'' : '','\sto\s':'','\sm\s':'', ',':'',  '\d*m':'','[0-9]*.[0-9]':'','[0-9]':'','\.|\'' :''}
df['Species '].replace(dic, regex = True, inplace= True)
dic1 ={'involveent|invovleent':'involvement','confired':'confirmed'}
df['Species '].replace(dic1, regex = True, inplace= True)


In [27]:
####Fatal Rellenamos cuando aparece Fatal en Injury y N cuando no aparece la palabra fatal
df['Fatal (Y/N)']= np.where(df['Injury'].str.contains('FATAL', flags=re.IGNORECASE, regex=True),  'Y', 'N')


In [28]:
####When we dont know the name we will call males John Doe and females Jane Doe
df['Name']= df['Name'].str.replace('female','Jane Doe')
df['Name']= df['Name'].str.replace(r'male|boy|men','John Doe',regex=True)


In [29]:
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula
6302,2018.06.25,25-Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
6301,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
6300,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
6299,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,John Doe,M,NaN,Minor injury to lower leg,N,NaN,shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
6298,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,Tiger shark,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,John Doe,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
4,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
3,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
2,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ÂºN, 79ÂºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [31]:
df.to_csv('attacks_new.csv', encoding = 'latin-1', sep=',')